In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [8]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [9]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']}



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)




Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [15]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.8571995513760676


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[77  8]
 [11 38]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.88      0.91      0.89        85
           1       0.83      0.78      0.80        49

    accuracy                           0.86       134
   macro avg       0.85      0.84      0.85       134
weighted avg       0.86      0.86      0.86       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


np.float64(0.8406962785114046)

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001167,0.000299,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000981,0.000143,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.005438,0.002410,0.010353,0.002917,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.888107,0.851527,0.775815,0.847020,0.833447,0.039097,3
3,0.005411,0.002401,0.008465,0.004333,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.814815,0.828959,0.814409,0.851527,0.850543,0.832051,0.016366,4
4,0.005221,0.001836,0.009579,0.004621,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.813511,0.815056,0.813179,0.867097,0.831197,0.022095,5
5,0.005518,0.001509,0.011164,0.003200,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.787654,0.790949,0.775815,0.905069,0.850543,0.822006,0.048974,8
6,0.001157,0.000405,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000988,0.000216,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.004693,0.001444,0.010967,0.002617,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.852030,0.833323,0.905069,0.773585,0.833754,0.044454,2
9,0.002473,0.000613,0.004019,0.000224,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.847141,0.852030,0.814409,0.850543,0.766556,0.826136,0.032847,6


In [30]:
age_input=float(input("Age:"))
estimated_salary_input=float(input("EstimatedSalary:"))
gender_male_input=int(input("Gender Male (0 for Female, 1 for Male):"))

Age:33
EstimatedSalary:3000
Gender Male (0 for Female, 1 for Male):1


In [31]:
# Prepare the input for prediction with the correct features and order
input_features = [[age_input, estimated_salary_input, gender_male_input]]

# Convert the input to a DataFrame with feature names
input_df = pd.DataFrame(input_features, columns=['Age', 'EstimatedSalary', 'Gender_Male'])

# Scale the input features using the same scaler used for training
scaled_input = sc.transform(input_df)

# Make the prediction
Future_Prediction = grid.predict(scaled_input)

print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[0]


In [32]:
result=Future_Prediction

In [33]:
result

array([0])